Error Handling

Nodes are where you decide policy: how many retries, what fallback message, what error to surface.

In [ ]:
from langchain_core.messages import AIMessage

Paris always works

In [ ]:
def weather_paris(state: MyMessagesState):
    text = get_weather("Paris")
    return {"messages": [AIMessage(content=text)]}

1 - First attempt for London. If it succeeds, we’re done.

2 - If it fails, we retry once.

3 - If the retry fails again, we gracefully degrade with a clear placeholder.(so that the assistant doesn’t hallucinate)

In [ ]:
def weather_london(state: MyMessagesState):
    try:
        # First attempt
        text = get_weather("London")
        return {"messages": [AIMessage(content=text)]}
    except Exception as e:
        # Retry once
        try:
            text = get_weather("London")
            return {"messages": [AIMessage(content=text)]}
        except Exception:
            # Fallback if still failing
            return {"messages": [AIMessage(content="London weather is currently unavailable.")]}

Parallel Routing/Tool calls

In [ ]:
def parallel_weather_condition(state: MyMessagesState):
    last_human = ""
    for msg in reversed(state["messages"]):
        if getattr(msg, "type", "") == "human":
            last_human = (msg.content or "").lower()
            break

    targets = []
    if "paris" in last_human:
        targets.append("weather_paris")
    if "london" in last_human:
        targets.append("weather_london")

    if not targets:
        targets = ["weather_paris"]

    return targets

The join node needs to accept both real tool outputs and fallback messages

In [ ]:
def combine_weather(state: MyMessagesState):
    lines = []
    for m in state["messages"]:
        if isinstance(m, AIMessage) and isinstance(m.content, str):
            if "The weather in" in m.content or "unavailable" in m.content:
                lines.append(m.content)

    combined = " | ".join(lines) if lines else "No weather data found." #Deterministic join: We concatenate lines so learners can predict the behavior.
    return {"messages": [AIMessage(content=f"Combined: {combined}")]}

We can replace the simple deterministic join step with an LLM summarizer to enhance the final text without changing the overall workflow structure.

Graph WorkFlow

 START → LLM → conditional → parallel branches → join → END 

In [ ]:
from langgraph.graph import StateGraph, START, END

In [ ]:
builder = StateGraph(MyMessagesState)

In [ ]:
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("weather_paris", weather_paris)
builder.add_node("weather_london", weather_london)
builder.add_node("combine_weather", combine_weather)

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", parallel_weather_condition)
builder.add_edge("weather_paris", "combine_weather")
builder.add_edge("weather_london", "combine_weather")
builder.add_edge("combine_weather", END)

In [ ]:
graph = builder.compile()

Inference

In [ ]:
from langchain_core.messages import HumanMessage

print("\n================= Test 1: Paris AND London =================")
res = graph.invoke({"messages": [HumanMessage(content="What's the weather in Paris and London?")]})
for m in res["messages"]:
    m.pretty_print()

print("\n================= Test 2: Paris only =================")
res = graph.invoke({"messages": [HumanMessage(content="What's the weather in Paris?")]})
for m in res["messages"]:
    m.pretty_print()